In [ ]:
from google.colab import drive#On monte le drive pour charger les datasets depuis google drive (optionnel)
drive.mount('/content/gdrive/', force_remount=True)
%cd gdrive/MyDrive/

Mounted at /content/gdrive/
/content/gdrive/MyDrive


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
import re
import pywt

def calculate_features(df): #fonction de feature engineering appliquée sur le dataset d'entrainement et le dataset de test
  # Ajout manuel de features
  #autocor = []
  #idmax = []
  entropy = []
  #df['WaveletVar'] = df.apply(wavelet_variance, axis=1)
  for i in range(len(df.index)) :
    #autocor.append(df.iloc[i,0:2160].autocorr(1))
    #idmax.append((int)(re.search(r'\d+',df.iloc[i,0:2160].idxmax())[0]))
    entropy.append(sum([ p * math.log(p) / math.log(2.0) for p in df.iloc[i,0:2160]]))
  #df['autocor'] = autocor
  #df['idmax'] = idmax
  df['entropy'] = entropy

  # Ajout automatique de features
  df['mean'] = df.iloc[:, 0:2160].mean(axis=1)
  df['min'] = df.iloc[:,0:2160].min(axis=1)
  df['max'] = df.iloc[:,0:2160].max(axis=1)
  df['median'] = df.iloc[:, 0:2160].median(axis=1)
  df['std'] = df.iloc[:, 0:2160].std(axis=1)
  df['var'] = df.iloc[:, 0:2160].var(axis=1)
  df['skew'] = df.iloc[:, 0:2160].skew(axis=1)
  df['sem'] = df.iloc[:, 0:2160].sem(axis=1)

X = pd.read_csv("InputTrain.csv").iloc[:,2:]
y = pd.read_csv("StepOne_LabelTrain.csv").iloc[:,2:]
dfXTest = pd.read_csv("InputTest.csv").iloc[:,2:]

calculate_features(X)
calculate_features(dfXTest)
'''
def wavelet_variance(row):
    coeffs = pywt.wavedec(row, 'db4', level=5) # perform 5-level decomposition using db4 wavelet
    variances = [c.var() for c in coeffs] # calculate the variances of each detail coefficient
    
    return sum(variances) # return the sum of variances
'''




"\ndef wavelet_variance(row):\n    coeffs = pywt.wavedec(row, 'db4', level=5) # perform 5-level decomposition using db4 wavelet\n    variances = [c.var() for c in coeffs] # calculate the variances of each detail coefficient\n    \n    return sum(variances) # return the sum of variances\n"

In [ ]:
X.isna().sum().sum()
#On vérifie qu'il n'y a aucune valeur NaN dans le dataset (peut causer beaucoup de problèmes à l'entrainement)

0

In [ ]:
import xgboost as xgb
from sklearn.metrics import average_precision_score
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.4,random_state=42)

dtrain = xgb.DMatrix(X_train,label=y_train)
dtest = xgb.DMatrix(X_test,label=y_test)

model = xgb.XGBModel()
params = {
    'max_depth': 10,
    'eta': 0.1,
    'objective': 'binary:logistic',
    'eval_metric': 'error',
    'booster':'dart',
    'sample_type':'weighted',
    'normalize_type':'forest',
    'learning_rate':0.4,
    'tree_method':'gpu_hist',
    'n_jobs':500,
    'rate_drop':0.3,
    'skip_drop':0.2,

}

# Train model
evalist = [(dtrain, 'train'), (dtest, 'eval')]
model = xgb.train(params, dtrain,num_boost_round=200)

preds = model.predict(dtest) #Predictions
y_test = y_test.astype(float)
print("Precision = {}".format(average_precision_score(y_test.values, preds, average='macro')))
print("Precision = {}".format(average_precision_score(y_test.values, preds, average='weighted')))




In [ ]:
dFin = xgb.DMatrix(dfXTest)
preds_f = model.predict(dFin)

dfF = pd.DataFrame(preds_f, columns=["Washing Machine","Dishwasher","Tumble Dryer","Microwave","Kettle"])
dfF = dfF.rename_axis('Index')
dfF.to_csv("corr3.csv")